## Paso 2. Prepara los datos.
Realizamos el procesamiento de datos y la limpieza de los mismos

In [122]:
#cargar librerias
#Importar las librerias que usaremos en el proyecto
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import math

In [123]:
#cargamos los datos del dataset
df_games = pd.read_csv('df_games_p.csv')

In [124]:
#Vamos a darle un vistazo a la informacion del dataset usamos info()
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


### Anotaciones de la primera impresión del data
- Se ve de entrada que los nombres de las columnas estan en mayusculas y minusculas, vamos a cambiarlas a minusculas
- Vamos a cambiar los tipos de datos de la columna User_Score a float y el de la columna Year a int

In [125]:
#Revisamos si existen valores perdidos en el df
df_games.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
NA_sales              0
EU_sales              0
JP_sales              0
Other_sales           0
Critic_Score       8578
User_Score         6701
Rating             6766
dtype: int64

- Vemos que existen valores perdidos en la columna Year, Critic,User_score, Name y Genre
- Para los valores perdidos de la columna Year los vamos a eliminar ya que son pocos valores

In [126]:
#Visualizamos la informacion general del data con sample() para conocer la informacion que contiene
df_games.sample(10)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
8751,Bust-A-Move Bash!,Wii,2007.0,Puzzle,0.13,0.01,0.00,0.01,NaN,NaN,NaN
6137,Shrek the Third,Wii,2007.0,Action,0.25,0.01,0.00,0.02,57.0,6.7,E10+
9146,The Granstream Saga,PS,1997.0,Role-Playing,0.08,0.05,0.00,0.01,NaN,NaN,NaN
1284,Disney Sing It: Pop Hits,Wii,2009.0,Misc,1.07,0.28,0.00,0.13,NaN,tbd,E
6142,Jackass the Game,PS2,2007.0,Action,0.14,0.11,0.00,0.04,58.0,6.6,M
13861,BlazBlue: Chrono Phantasma Extend,PS3,2015.0,Action,0.00,0.00,0.04,0.00,NaN,8.3,T
10256,Shining Ark,PSP,2013.0,Role-Playing,0.00,0.00,0.11,0.00,NaN,NaN,NaN
5083,Kileak: The DNA Imperative,PS,1994.0,Shooter,0.08,0.06,0.21,0.02,NaN,NaN,NaN
2312,LEGO Pirates of the Caribbean: The Video Game,DS,2011.0,Action,0.34,0.46,0.00,0.10,69.0,tbd,E10+
5221,Spider-Man: Shattered Dimensions,X360,2010.0,Action,0.24,0.08,0.00,0.03,76.0,7.6,T


- Viendo los datos se ve que los datos de la columna Name igual vienen en mayusculas y minusculas y algunos signos de deseparación entre palabras vamos a quitarlas


In [127]:
#Vemos los valores unicos de la columna Platform y contamos cuantos valores son
print(df_games['Platform'].unique())

df_games['Platform'].nunique()

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']


31

- Vemos que tenemos 31 plataformas de video juegos
- Podemos agrupar los juegos con para reducir el numero de plataformas

In [128]:
#Detalle de la columna Year
df_games['Year_of_Release'].unique()

array([2006., 1985., 2008., 2009., 1996., 1989., 1984., 2005., 1999.,
       2007., 2010., 2013., 2004., 1990., 1988., 2002., 2001., 2011.,
       1998., 2015., 2012., 2014., 1992., 1997., 1993., 1994., 1982.,
       2016., 2003., 1986., 2000.,   nan, 1995., 1991., 1981., 1987.,
       1980., 1983.])

In [129]:
#Detalle de generos contamos los valores unicos de los juegos por genero
df_games['Genre'].value_counts()

Genre
Action          3369
Sports          2348
Misc            1750
Role-Playing    1498
Shooter         1323
Adventure       1303
Racing          1249
Platform         888
Simulation       873
Fighting         849
Strategy         683
Puzzle           580
Name: count, dtype: int64

In [130]:
df_games['Critic_Score'].unique()

array([76., nan, 82., 80., 89., 58., 87., 91., 61., 97., 95., 77., 88.,
       83., 94., 93., 85., 86., 98., 96., 90., 84., 73., 74., 78., 92.,
       71., 72., 68., 62., 49., 67., 81., 66., 56., 79., 70., 59., 64.,
       75., 60., 63., 69., 50., 25., 42., 44., 55., 48., 57., 29., 47.,
       65., 54., 20., 53., 37., 38., 33., 52., 30., 32., 43., 45., 51.,
       40., 46., 39., 34., 35., 41., 36., 28., 31., 27., 26., 19., 23.,
       24., 21., 17., 22., 13.])

- Vemos que los valores de la puntuacion de los criticos van del 0 al 100 

In [131]:
#vemos los valores unicos de la columna User_Score tenemos que existe un valor string llamado tdb
df_games['User_Score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

- Vemos que los valores del puntuacion de los usuarios va de 1 a 10 a diferencia que la de los criticos, vamos a ponerlos de 0 100 tambien. 

## Paso 2. Procesamiento de datos 


Ya revisando los datos del dataset comenzamos a preparar y pulir los datos.
Se observa que el nombre de las columnas estan en mayusculas y minisculas. Y que algunos tipos de datos de las columnas estan erroneos vamos corregirlos.

In [132]:
#2.1. Cambiar a minusculas el nombre de las columnas
df_games.columns = [x.lower() for x in df_games.columns]

In [133]:
#vemos si se cambiaron a minusculas
df_games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

Le damos snake cases a la columna name

In [134]:
#cambios a minusculas el nombre de los juegos  
df_games['name'] = df_games['name'].str.lower()
#Quitamos puntos o comas
df_games['name'] = df_games['name'].str.replace(".","").replace(",","")


In [135]:
df_games['name']

0                           wii sports
1                     super mario bros
2                       mario kart wii
3                    wii sports resort
4             pokemon red/pokemon blue
                     ...              
16710    samurai warriors: sanada maru
16711                 lma manager 2007
16712          haitaka no psychedelica
16713                 spirits & spells
16714              winning post 8 2016
Name: name, Length: 16715, dtype: object

In [136]:
#vemos los 2 valores perdidos de la columna name
df_games[df_games['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


In [137]:
#vamos a renombrar esos valors perdidos por no_definido
df_games['name'] = df_games['name'].fillna('no_definido')

Vamos agrupar los nombres de las plataformas ya que son muchas y solo tendremos 5 grandes Grupos

In [138]:
#Agrupamos los nombres de las columnas
#Hacemos una funcion para  agrupar
def agrupar_plat (x):
    if x in ['Wii','GB','GC','DS','WiiU','NES','SNES','GBA','3DS','N64']:
        return 'nintendo'
    elif x in ['X360','XB','XOne']:
        return 'xbox'
    elif x in ['PS3','PS2','PS','PSP','PS4','PSV']:
        return 'play'
    elif x in ['PC','PCFX']:
        return 'pc'
    else:
        return 'otros'


In [139]:
#Aplicamos la funcion para crear la columna agrupacion
df_games['group_plat'] = df_games['platform'].apply(agrupar_plat)

In [140]:
df_games.sample(5)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,group_plat
2761,midnight club 3: dub edition,XB,2005.0,Racing,0.61,0.10,0.00,0.03,84.0,7.4,E10+,xbox
14302,casino chaos with las vegas players collection,PC,2011.0,Misc,0.03,0.00,0.00,0.00,NaN,tbd,T,pc
6340,stuntman: ignition,PS2,2007.0,Racing,0.13,0.10,0.00,0.03,71.0,6.4,T,play
9207,transformers: devastation,XOne,2015.0,Action,0.09,0.04,0.00,0.01,75.0,7.9,T,xbox
5894,dynasty warriors 8: xtreme legends,PS4,2014.0,Action,0.09,0.11,0.06,0.04,NaN,NaN,NaN,play


In [141]:
df_games['group_plat'].value_counts()

group_plat
play        6720
nintendo    6270
xbox        2333
pc           975
otros        417
Name: count, dtype: int64

#### 2.4.3 Presta atención a la abreviatura TDB: Significa "to be determined" (a determinar). Especificar como piensas manejar estos casos.

Para todos los casos "tbd", vamos a cambiarlos por un valor perdido NAN, para con esto poder cambiar el tipo de dato de la columna a float

In [142]:
#Cambiar el caso de tbd por un valor numerico perdido con np.nan
df_games.loc[df_games['user_score']== 'tbd','user_score'] = np.nan

In [143]:
#vamos a cambiar el tipo de datos a float y lo multiplicamos por 10 para que este homogeneo con los datos de critic_score
df_games['user_score'] = df_games['user_score'].astype(float)*10

In [144]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16715 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
 11  group_plat       16715 non-null  object 
dtypes: float64(7), object(5)
memory usage: 1.5+ MB


In [145]:
df_games.sample(5)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,group_plat
13068,dodonpachi resurrection,X360,2010.0,Shooter,0.00,0.04,0.00,0.01,77.0,86.0,NaN,xbox
2878,scooby-doo 2: monsters unleashed,GBA,2004.0,Action,0.51,0.19,0.00,0.01,46.0,NaN,E,nintendo
172,dr mario,GB,1989.0,Puzzle,2.18,0.96,2.00,0.20,NaN,NaN,NaN,nintendo
8889,the incredibles: rise of the underminer,DS,2005.0,Action,0.13,0.01,0.00,0.01,54.0,NaN,E10+,nintendo
12062,trick ds-han: kakushi kami no sumukan,DS,2010.0,Misc,0.00,0.00,0.07,0.00,NaN,NaN,NaN,nintendo


In [146]:
#Revisar si hay duplicados 
df_games[df_games[['name','platform','year_of_release']].duplicated()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,group_plat
14244,no_definido,GEN,1993.0,NaN,0.0,0.00,0.03,0.0,NaN,NaN,NaN,otros
16230,madden nfl 13,PS3,2012.0,Sports,0.0,0.01,0.00,0.0,83.0,55.0,E,play


In [147]:
#Eliminamos duplicados con dropduplicates y subset
df_games = df_games.drop_duplicates(subset= ['name','platform','year_of_release']).reset_index(drop=True)

In [148]:
df_games.isna().sum()

name                  0
platform              0
year_of_release     269
genre                 1
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8577
user_score         9124
rating             6765
group_plat            0
dtype: int64

Se eliminaron los valores ausentes de la columna year_of_release ya que estos corresponden al 2% de los valores se uso el metodo dropna junto con subset y se selecciono la columna year_of_release para eliminar los valores perdidos de esta columna.

In [149]:
#vamos a eliminar los valores perdidos de year of ya que representa el 2 % de los datos
df_games_clean = df_games.dropna(subset= ['year_of_release'])

In [150]:
#Vamos a convertir el tipo de dato de la columna year_of_release a int
df_games_clean['year_of_release'] = df_games_clean['year_of_release'].astype(int)

C:\Users\sistemas\AppData\Local\Temp\ipykernel_8180\2433759101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_clean['year_of_release'] = df_games_clean['year_of_release'].astype(int)


#### 2.5. Calcula las ventas totales(La suma de las ventas en todas las regiones). Para cada juego y coloca estos valores en una columna separada, 

In [151]:
#Creamos una nueva columna llamada ventas_totales con la suma de las ventas de las 4 columnas na_sales,eu_sales,jp_sales y other_sales
df_games_clean['ventas_totales'] =  df_games_clean['na_sales'] + df_games_clean['eu_sales']+ df_games_clean['jp_sales'] + df_games_clean['other_sales']
df_games_clean['ventas_totales'] = round(df_games_clean['ventas_totales'],2)

C:\Users\sistemas\AppData\Local\Temp\ipykernel_8180\181612390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_clean['ventas_totales'] =  df_games_clean['na_sales'] + df_games_clean['eu_sales']+ df_games_clean['jp_sales'] + df_games_clean['other_sales']
C:\Users\sistemas\AppData\Local\Temp\ipykernel_8180\181612390.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_clean['ventas_totales'] = round(df_games_clean['ventas_totales'],2)


In [152]:
#guardamos el dataset con los valores limpios
df_games_clean.to_csv('df_games_clean.csv',index=False)